# Cell 1 – Imports & paths

In [ ]:
import os
from collections import defaultdict

import cv2
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Video

SEQ = "seq_02"

# Base directory for your sequence 1 data
BASE_DIR = f"34759_final_project_rect/{SEQ}"

LEFT_DIR  = os.path.join(BASE_DIR, "image_02", "data")  # left images
RIGHT_DIR = os.path.join(BASE_DIR, "image_03", "data")  # right images
LABELS_TXT = os.path.join(BASE_DIR, "labels.txt")

print("Left images dir :", LEFT_DIR)
print("Right images dir:", RIGHT_DIR)
print("Labels file     :", LABELS_TXT)


# Cell 2 – Load labels into a dict: frame_id -> list of objects

In [ ]:
# Cell 2: parse labels.txt into a dictionary

def load_labels(labels_path):
    """
    Returns:
        dict: frame_id (int) -> list of dicts with keys:
              'track_id', 'type', 'truncated', 'occluded', 'alpha',
              'bbox', 'dimensions', 'location', 'rotation_y'
    """
    annotations = defaultdict(list)

    with open(labels_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split()
            # According to description:
            # frame, track_id, type, truncated, occluded, alpha,
            # bbox_l, bbox_t, bbox_r, bbox_b,
            # h, w, l,
            # x, y, z,
            # rotation_y,
            # [optional score]

            frame      = int(parts[0])
            track_id   = int(parts[1])
            obj_type   = parts[2]
            truncated  = float(parts[3])
            occluded   = int(parts[4])
            alpha      = float(parts[5])

            bbox = list(map(float, parts[6:10]))          # [l, t, r, b]
            dims = list(map(float, parts[10:13]))         # [h, w, l]
            loc  = list(map(float, parts[13:16]))         # [x, y, z]
            rotation_y = float(parts[16])

            ann = {
                "track_id": track_id,
                "type": obj_type,
                "truncated": truncated,
                "occluded": occluded,
                "alpha": alpha,
                "bbox": bbox,
                "dimensions": dims,
                "location": loc,
                "rotation_y": rotation_y,
            }

            # If score is present (for detections) you could also store it:
            if len(parts) > 17:
                ann["score"] = float(parts[17])
            
            if occluded > 0:
                annotations[frame].append(ann)

    return annotations

annotations_by_frame = load_labels(LABELS_TXT)
print(f"Loaded annotations for {len(annotations_by_frame)} frames.")
print("Example frame ids:", list(sorted(annotations_by_frame.keys()))[:10])


# Cell 3 – List & index images by frame

In [ ]:
# Cell 3: load sorted list of left and right images

def sorted_image_paths(img_dir):
    files = [f for f in os.listdir(img_dir) if f.lower().endswith(".png")]
    files = sorted(files)
    return [os.path.join(img_dir, f) for f in files]

left_image_paths  = sorted_image_paths(LEFT_DIR)
right_image_paths = sorted_image_paths(RIGHT_DIR)

num_frames = min(len(left_image_paths), len(right_image_paths))
print(f"Found {num_frames} frames.")

# Simple helper: get image for a given frame id
def load_frame_images(frame_id):
    if frame_id < 0 or frame_id >= num_frames:
        raise IndexError(f"Frame id {frame_id} out of range (0..{num_frames-1})")

    left_img  = cv2.imread(left_image_paths[frame_id])
    right_img = cv2.imread(right_image_paths[frame_id])
    return left_img, right_img


# Cell 4 – Drawing helper (bounding boxes, labels, coordinates)

In [ ]:
# Cell 4: function to overlay annotations on an image

def draw_annotations(image, annotations):
    """
    Draws bounding boxes, labels, and 3D coordinates on a copy of `image`.
    Returns the annotated image (BGR).
    """
    img = image.copy()

    for ann in annotations:
        l, t, r, b = ann["bbox"]
        obj_type = ann["type"]
        track_id = ann["track_id"]
        x, y, z = ann["location"]

        # Convert to int pixel coords
        pt1 = (int(l), int(t))
        pt2 = (int(r), int(b))

        # Box
        cv2.rectangle(img, pt1, pt2, (0, 255, 0), 2)

        # First line: class + id
        label_1 = f"{obj_type} #{track_id}"
        # Second line: x,z (you can include y as well)
        label_2 = f"x={x:.1f}m, y={y:.1f}m, z={z:.1f}m"

        # Put text slightly above box
        y0 = max(int(t) - 25, 0)
        cv2.putText(img, label_1, (int(l), y0),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

        y1 = max(int(t) - 5, 0)
        cv2.putText(img, label_2, (int(l), y1),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

    return img


# Cell 5 – Show a few sample frames with overlays

In [ ]:
# Cell 5: visualize some sample frames with bounding boxes

sample_frames = sorted(list(annotations_by_frame.keys()))[:3]  # first 3 frames with objects

plt.figure(figsize=(15, 5 * len(sample_frames)))

for i, frame_id in enumerate(sample_frames):
    left_img, _ = load_frame_images(frame_id)
    anns = annotations_by_frame.get(frame_id, [])

    left_img_annotated = draw_annotations(left_img, anns)
    # Convert BGR -> RGB for matplotlib
    left_rgb = cv2.cvtColor(left_img_annotated, cv2.COLOR_BGR2RGB)

    plt.subplot(len(sample_frames), 1, i + 1)
    plt.imshow(left_rgb)
    plt.title(f"Frame {frame_id} with {len(anns)} objects")
    plt.axis("off")

plt.tight_layout()
plt.show()


# Cell 6 – Create a video with all frames + overlays

In [ ]:
# Cell 6: create a video with annotations over all frames

output_video_path = os.path.join(f"{SEQ}_annotated.mp4")
fps = 10  # change if you like

# Read the first frame to get size
first_left, _ = load_frame_images(0)
height, width = first_left.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

for frame_id in range(num_frames):
    left_img, _ = load_frame_images(frame_id)
    anns = annotations_by_frame.get(frame_id, [])
    frame_annotated = draw_annotations(left_img, anns)
    video_writer.write(frame_annotated)

video_writer.release()
print("Saved video to:", output_video_path)
